In [ ]:
import re
import sys

import time
import os 
from xml.dom import minidom
import json 


projectName = input("Enter a name for the Project: ")
if not re.match("^[a-zA-Z]+$", projectName):
    print("Error! Only letters a-z allowed!")
    sys.exit()
elif len(projectName) > 15:
    print("Error! Only 15 characters allowed!")
    sys.exit()


numLabels=int(input("Enter the number of fields for the Project: "))

labelsList=[]
for i in range(numLabels):
    
    labelName = input("Enter a name for the Label"+str(i+1)+": ")
    if not re.match("^[a-zA-Z]+$", projectName):
        print("Error! Only letters a-z allowed!")
        sys.exit()
    elif len(projectName) > 15:
        print("Error! Only 15 characters allowed!")
        sys.exit()
    
    labelType= input("Input a type for the label: [Person,Date,CovidResult]")
    
    
    labelsList.append({'labelName':labelName,'labelType':labelType})





In [ ]:
#write the labels to a file

with open("data/labels.json", "w") as outfile:
    json.dump(labelsList, outfile)


In [ ]:
#create LS compatiable config.xml from the labels
#config template
'''
<View>
  <Image name="image" value="$ocr"/>

  <Labels name="label" toName="image">
    <Label value="PatientName" background="green"/>
    <Label value="ABHAID" background="blue"/>
    <Label value="CovidResult" background="blue"/>
  </Labels>

  <Rectangle name="bbox" toName="image" strokeWidth="3"/>
  

  <TextArea name="transcription" toName="image"
            editable="true"
            perRegion="true"
            maxSubmissions="1"
            rows="1"
            
            displayMode="region-list"
            />
</View>
'''
root = minidom.Document()
xml = root.createElement('View') 
root.appendChild(xml)

#image element for root View
imageChild = root.createElement('Image')
imageChild.setAttribute('name', 'image')
imageChild.setAttribute('value', '$imagepath')
xml.appendChild(imageChild)

#labels elements for root View
labelsChild=root.createElement('Labels')
labelsChild.setAttribute('name', 'label')
labelsChild.setAttribute('toName', 'image')
xml.appendChild(labelsChild)

for labels in labelsList:
    labelValueChild=root.createElement('Label')
    labelValueChild.setAttribute('value', labels['labelName'])
    labelsChild.appendChild(labelValueChild)

#TextArea element for root View
TextArea = root.createElement('TextArea')
TextArea.setAttribute('name', 'transcription')
TextArea.setAttribute('toName', 'image')
TextArea.setAttribute('editable', 'true')
TextArea.setAttribute('perRegion', 'true')
TextArea.setAttribute('maxSubmissions', '2')
TextArea.setAttribute('rows', '1')
TextArea.setAttribute('displayMode', 'region-list')
xml.appendChild(TextArea)

#Rectangle element for root View
Rectangle = root.createElement('Rectangle')
Rectangle.setAttribute('name', 'bbox')
Rectangle.setAttribute('toName', 'image')
Rectangle.setAttribute('strokeWidth', '3')



xml.appendChild(Rectangle)

xml_str = root.toprettyxml(indent ="\n") 

with open("data/config.xml", "w") as outfile:
    outfile.write(xml_str)

 

In [ ]:
#check LS connection
# Import the SDK and the client module
from label_studio_sdk import Client
from label_studio_sdk import Project
from label_studio_sdk.data_manager import Column, Filters, Operator, Type
from datetime import datetime


# Define the URL where Label Studio is accessible and the API key for your user account
LABEL_STUDIO_URL = 'http://localhost:8080'
API_KEY = '50b414a21224bb7e89be706c93a7b25df6f6b367'


# Connect to the Label Studio API and check the connection
ls = Client(url=LABEL_STUDIO_URL, api_key=API_KEY)
ls.check_connection()


In [ ]:
#create the project
project = ls.start_project(
    title=projectName,
    label_config=xml_str
)